# Object Detect & Find - Live Demo

In this notebook we'll show how you can seek & find an object with JetBot!  We'll use a pre-trained quantized COCO SSD MobileNet v1 model.
https://www.tensorflow.org/lite/models/object_detection/overview
#### Uses and limitations
The object detection model we provide can identify and locate up to 10 objects in an image. It is trained to recognize 80 classes of object.
If you want to train a model to recognize new classes, see Customize model(https://www.tensorflow.org/lite/models/object_detection/overview#customize_model).

### Compute detections on single camera image

In [4]:
from jetbot import Camera

camera = Camera.instance(width=300, height=300)

## TFLite
#### Input
The model takes an image as input. The expected image is 300x300 pixels, with three channels (red, blue, and green) per pixel. This should be fed to the model as a flattened buffer of 270,000 byte values (300x300x3). Since the model is quantized, each value should be a single byte representing a value between 0 and 255.
#### Output
The model outputs four arrays, mapped to the indices 0-4. Arrays 0, 1, and 2 describe 10 detected objects, with one element in each array corresponding to each object. There will always be 10 objects detected.
0 	Locations,  1 	Classes, 2 	Scores, 3 	Number and detections

## TensorRT
Internally, the ``ObjectDetector`` class uses the TensorRT Python API to execute the engine that we provide.  It also takes care of preprocessing the input to the neural network, as well as parsing the detected objects.  Right now it will only work for engines created using the ``jetbot.ssd_tensorrt`` package. That package has the utilities for converting the model from the TensorFlow object detection API to an optimized TensorRT engine.

Next, let's initialize our camera.  Our detector takes 300x300 pixel input, so we'll set this when creating the camera.

> Internally, the Camera class uses GStreamer to take advantage of Jetson Nano's Image Signal Processor (ISP).  This is super fast and offloads
> a lot of the resizing computation from the CPU. 

In [1]:
from tflite_runtime.interpreter import Interpreter
import re
import numpy as np

def load_labels(path):
  """Loads the labels file. Supports files with or without index numbers."""
  with open(path, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    labels = {}
    for row_number, content in enumerate(lines):
      pair = re.split(r'[:\s]+', content.strip(), maxsplit=1)
      if len(pair) == 2 and pair[0].strip().isdigit():
        labels[int(pair[0])] = pair[1].strip()
      
      else:
        labels[row_number] = pair[0].strip()
  
  return labels

# labels = load_labels('labelmap.txt')
with open('../object_detection/labelmap.txt', 'r') as F:
     class_names = F.readlines()
        
interpreter = Interpreter('../object_detection/detect.tflite')
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

Now, let's execute our network using some camera input.  By default the ``ObjectDetector`` class expects ``bgr8`` format that the camera produces.  However,
you could override the default pre-processing function if your input is in a different format.

In [1]:
from jetbot import ObjectDetector

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

According to pretrained model, it may have a pre-processing different format like float32(uint8) or pixel size.

In [ ]:
import cv2
import numpy as np

def bgr8_to_tf_input(img):
    x = cv2.resize(img, (300, 300))
    x = x[:, :, [2,1,0]]  # BGR -> RGB
    x = np.expand_dims(x, axis=0) # 3->4
    return x

img = bgr8_to_tf_input(camera.value)
# img = cv2.imread('chairs.jpg')
test_x = bgr8_to_tf_input(img)

## TensorRT model
If there are any COCO objects in the camera's field of view, they should now be stored in the ``detections`` variable.

In [6]:
import time

start_time = time.monotonic()

detections = model(camera.value)

elapsed_ms = (time.monotonic() - start_time) * 1000
print('elapsed time={}'.format(elapsed_ms))

print(detections)

elapsed time=265.81469399934576
[[{'label': 72, 'confidence': 0.9341553449630737, 'bbox': [0.5708431601524353, 0.07103028893470764, 0.9479846358299255, 0.5153477191925049]}, {'label': 1, 'confidence': 0.7708099484443665, 'bbox': [0.2301267385482788, 0.13638506829738617, 0.3498685359954834, 0.3965451121330261]}, {'label': 73, 'confidence': 0.5355843305587769, 'bbox': [0.334503173828125, 0.22310680150985718, 0.5233591198921204, 0.4008357524871826]}, {'label': 73, 'confidence': 0.3796083629131317, 'bbox': [0.5769811868667603, 0.07369008660316467, 0.944791316986084, 0.5196888446807861]}]]


## TFLite model
Set an image to Interpreter of TFLite runtime. Run detect interpreter from invoke API.
Allocate detect results to any array.

In [3]:
import time

start_time = time.monotonic()

interpreter.set_tensor(input_details[0]['index'], test_x)

interpreter.invoke()

elapsed_ms = (time.monotonic() - start_time) * 1000
print('elapsed time={}'.format(elapsed_ms))

bbox = interpreter.get_tensor(output_details[0]['index'])  # Locations (Top, Left, Bottom, Right)
class_label = interpreter.get_tensor(output_details[1]['index'])  # Classes (0=Person or 0=unlabeled)
score = interpreter.get_tensor(output_details[2]['index'])  # Scores
total_detects = interpreter.get_tensor(output_details[3]['index'])  # Number of detections


elapsed time=547.0000000022992


### Display detections in text area

We'll use the code below to print out the detected objects.

In [6]:
from IPython.display import display
import ipywidgets.widgets as widgets

NumofDetects = int(total_detects[0])

dstimg = bytes(cv2.resize(img, (300, 300)))
              
for i in range(NumofDetects):
    prob = score[0, i]
    class_name = class_names[class_label[0, i].astype(int)+ 1].rstrip()
    (top, left, bottom, right) = bbox[0, i] * 300
    center_x = (right + left) / 2.0 - 0.5
    center_y = (bottom + top) / 2.0 - 0.5
    dis= np.sqrt((150-center_x)**2 + (300-center_y)**2)
    if prob >= 0.65:
        print("Location=({},{})-({},{})".format(int(left), int(top), int(right), int(bottom)))
        print("Class={}".format(class_name))
        print("Probability={}".format(prob))
        print("Dis={}".format(dis))


Location=(198,120)-(271,244)
Class=chair
Probability=0.7578125
Dis=145.09704723145617
Location=(84,119)-(186,292)
Class=chair
Probability=0.7578125
Dis=95.95022873285181
Location=(31,84)-(94,207)
Class=chair
Probability=0.65625
Dis=177.59840900673376


You should see the label, confidence, and bounding box of each object detected in each image.  There's only one image (our camera) in this example. 


To print just the first object detected in the first image, we could call the following

> This may throw an error if no objects are detected

In [40]:
cv2.imshow("test image", img)
cv2.moveWindow("test image", 0, 0)
cv2.waitKey(0)
cv2.destroyAllWindows()

# image_number = 0
# object_number = 0
# print(detections[image_number][object_number])

### Control robot to follow central object

Now we want our robot to follow an object of the specified class.  To do this we'll do the following

1.  Detect objects matching the specified class
2.  Select object closest to center of camera's field of vision, this is the 'target' object
3.  Steer robot towards target object, otherwise wander
4.  If we're blocked by an obstacle, turn left

We'll also create some widgets that we'll use to control the target object label, the robot speed, and
a "turn gain", that will control how fast the robot turns based off the distance between the target object
and the center of the robot's field of view. 


First, let's load our collision detection model.  The pre-trained model is stored in this directory as a convenience, but if you followed
the collision avoidance example you may want to use that model if it's better tuned for your robot's environment.

In [ ]:
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np

collision_model = torchvision.models.alexnet(pretrained=False)
collision_model.classifier[6] = torch.nn.Linear(collision_model.classifier[6].in_features, 2)
collision_model.load_state_dict(torch.load('../collision_avoidance/best_model.pth'))
device = torch.device('cuda')
collision_model = collision_model.to(device)

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

Great, now let's initialize our robot so we can control the motors.

In [ ]:
from jetbot import Robot

robot = Robot()

Finally, let's display all the control widgets and connect the network execution function to the camera updates.

In [8]:
# widget generation
blocked_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blocked')
distance_widget = widgets.FloatSlider(value=50.0, min=0.0, max=300.0, description='Dis threshold')
image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=1, description='tracked label')
speed_widget = widgets.FloatSlider(value=0.4, min=0.0, max=1.0, description='speed')
turn_gain_widget = widgets.FloatSlider(value=0.8, min=0.0, max=2.0, description='turn gain')
detections_widget = widgets.Textarea()
detections_widget.value = str(class_names[label_widget.value].rstrip())
#display(detections_widget)

display(widgets.VBox([
    widgets.HBox([image_widget, blocked_widget]),
    label_widget,
    detections_widget,
    speed_widget,
    turn_gain_widget,
    distance_widget
]))

width = int(image_widget.width)
height = int(image_widget.height)
# image_widget.value = dstimg

In [ ]:
from jetbot import bgr8_to_jpeg

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection
        
def execute(change):
    image = change['new']
    
    # execute collision model to determine if blocked
    collision_output = collision_model(preprocess(image)).detach().cpu()
    prob_blocked = float(F.softmax(collision_output.flatten(), dim=0)[0])
    blocked_widget.value = prob_blocked
    
    # turn left if blocked
    if prob_blocked > 0.5:
        robot.left(0.3)
        image_widget.value = bgr8_to_jpeg(image)
        return
        
    # compute all detected objects
    detections = model(image)
    
    # draw all detections on image
    for det in detections[0]:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
    
    # select detections that match selected class label
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
    # get detection closest to center of field of view and draw it
    det = closest_detection(matching_detections)
    if det is not None:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 5)
       
        
    # otherwise go forward if no target detected
    if det is None:
        robot.forward(float(speed_widget.value))
        
    # otherwsie steer towards target
    else:
        # move robot forward and steer proportional target's x-distance from center
        center = detection_center(det)
        target_dis = norm(center)
        if target_dis <= int(distance_widget.value):
            robot.stop() 
        else:
            robot.set_motors(
                float(speed_widget.value + turn_gain_widget.value * center[0]),
                float(speed_widget.value - turn_gain_widget.value * center[0])
            )
    
    # update image widget
    image_widget.value = bgr8_to_jpeg(image)
    
execute({'new': camera.value})

Call the block below to connect the execute function to each camera frame update.

In [ ]:
camera.unobserve_all()
camera.observe(execute, names='value')

Awesome!  If the robot is not blocked you should see boxes drawn around the detected objects in blue.  The target object (which the robot follows) will be displayed in green.

The robot should steer towards the target when it is detected.  If it is blocked by an object it will simply turn left.

You can call the code block below to manually disconnect the processing from the camera and stop the robot.

In [ ]:
import time

camera.unobserve_all()
time.sleep(1.0)
robot.stop()